# From python to Spark (PySpark)

PySpark is a Python API for Apache Spark

Getting started with Spark for Python programmers is particularly easy if one is using a certain programming style: 
maps, filters and lambda functions.

# Lambda functions in Python

Python supports the creation of anonymous functions (i.e. functions that are not bound to a name) at runtime, using a construct called "lambda".

Sometimes you need to pass a function as an argument, or you want to do a short but complex operation multiple times. You could define your function the normal way, or you could make a lambda function, a mini-function that returns the result of a single expression. The two definitions are completely identical:

In [ ]:
##traditional named function
def add(a,b): return a+b

##lambda function
add2 = lambda a,b: a+b

The advantage of the lambda function is that it is in itself an expression, and can be used inside another statement. Here's an example using the map function, which calls a function on every element in a list, and returns a list of the results:

In [ ]:
squares = map(lambda a: a*a, [1,2,3,4,5])

# Exercise 1: mapping the list in Python

Suppose you need to perform a transformation on a list of element. For instance, to calculate a square of each element of the list. One way to write this in Python would be as follows: 

In [ ]:
numbers = [1,2,3,4,5]
squares = []
for number in numbers:
    squares.append(number*number)
    # Now, squares should have [1,4,9,16,25]
print "List of squares: ", squares

## Pythonic way

Python provides a few ways to re-write the same piece of code in a more compact form: list comprehensions and with the map. Python programmers who do not use Spark typically prefer the list comprehensions to the map. But using the map is what allows you to adjust to Spark way of programming the easiest:

In [ ]:
numbers = [1,2,3,4,5]
squares = map(lambda x: x*x, numbers)
#Now, squares should have [1,4,9,16,25]
print "List of squares: ", squares

# Exercise 2: filtering the list in Python

What if you're more interested in filtering the list? Say you want to remove every element with a value equal to or greater than 4? (Okay, so the examples aren't very realistic. Whatever...) A Python neophyte might write:

In [ ]:
numbers = [1,2,3,4,5]
numbers_under_4 = []
for number in numbers:
    if number < 4:
        numbers_under_4.append(number)
        # Now, numbers_under_4 contains [1,4,9]
print "Numbers under 4 only: ",numbers_under_4

You could reduce the size of the code with the filter function:

In [ ]:
numbers = [1,2,3,4,5]
numbers_under_4 = filter(lambda x: x < 4, numbers)
# Now, numbers_under_4 contains [1,2,3]
print "Numbers under 4 only: ",numbers_under_4

# Exercise 3: Pandas data structures and functionality

Pandas is Python's answer to R.  It's a good tool for small(ish) data analysis -- i.e., when everything fits into memory. The basic new "noun" in pandas is the **data frame**. As a part of pre-exercises, you have received an iPython notebook with some Pandas case study. 

It's like a table, with rows and columns (e.g., as in SQL).  Except:
  - The rows can be indexed by something interesting (there is special support for labels like categorical and timeseries data).  This is especially useful when you have timeseries data with potentially missing data points.
  - Cells can store Python objects. (Like in SQL, columns are homogeneous.)
  - Instead of "NULL", the name for a non-existent value is "NA".  Unlike R, Python's data frames only support NAs in columns of some data types (basically: floating point numbers and 'objects') -- but this is mostly a non-issue (because it will "up-cast" integers to float64, etc.)
  
Pandas provides a "batteries-included" basic data analysis:
  - **Loading data:** `read_csv`, `read_table`, `read_sql`, and `read_html`
  - **Selection, filtering, and aggregation** (i.e., SQL-type operations): There's a special syntax for `SELECT`ing.  There's the `merge` method for `JOIN`ing.  There's also an easy syntax for what in SQL is a mouthful: Creating a new column whose value is computed from other column -- with the bonus that now the computations can use the full power of Python (though it might be faster if it didn't).
  - **"Pivot table" style aggregation**: If you're an Excel cognosceti, you may appreciate this.
  - **NA handling**: Like R's data frames, there is good support for transforming NA values with default values / averaging tricks / etc.
  - **Basic statistics:** e.g. `mean`, `median`, `max`, `min`, and the convenient `describe`.
  - **Plugging into more advanced analytics:** Okay, this isn't batteries included.  But still, it plays reasonably with `sklearn`.
  - **Visualization:** For instance `plot` and `hist`.
  
  
## Map and filter in Pandas:


In [1]:
import pandas as pd

names =["State_Code", "County_Code", "Census_Tract_Number", "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL", "AMT_FHA", "PCT_AMT_FHA"]
df = pd.read_csv('../preexercise/data/fha_by_tract.csv', names=names)  ## Loading a CSV file, without a header (so we have to provide field names)

df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
0,8,75,NaN,1,1,100,258,258,100
1,28,49,103.01,1,1,100,71,71,100
2,40,3,NaN,1,1,100,215,215,100
3,39,113,603.00,3,3,100,206,206,100
4,12,105,124.04,2,2,100,303,303,100


In [2]:
df["State_Code2"] = df["State_Code"].apply(lambda x: x+1)

df["State_Code2"].head()

0     9
1    29
2    41
3    40
4    13
Name: State_Code2, dtype: float64

In [8]:
df = df[df['County_Code'] > 75]

df.head()

,State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,State_Code2,County_Code2
3,39,113,603.00,3,3,100,206,206,100,40,39
4,12,105,124.04,2,2,100,303,303,100,13,12
5,12,86,9808.00,1,1,100,188,188,100,13,12
7,12,103,207.00,2,2,100,100,100,100,13,12
8,36,119,30.00,1,1,100,354,354,100,37,36


# Exercise 4: from Python to PySpark

Say I want to map and filter a list at the same time. In other words, I'd like to see the square of each element in the list where said element is under 4. Once more, the Python neophyte way:


In [ ]:
numbers = [1,2,3,4,5]
squares = []
for number in numbers:
    if number < 4:
        squares.append(number*number)
print squares

Before re-writing it in PySpark, re-write it using map and filter expressions:

In [ ]:
numbers = [1,2,3,4,5]
squares = map(lambda x: x*x, filter(lambda x: x < 4, numbers))
print squares

Now do with PySpark

In [ ]:
#I do not need to create the Spark Context in the notebook, but you do...
#sc = SparkContext("My First App")
numbers_rdd = sc.parallelize(numbers)
squares_rdd = numbers_rdd.filter(lambda x: x < 4).map(lambda x: x*x)
print squares_rdd.collect()

# Transformations in Python and PySpark (quick look forward)

It is Pythonic to operate on lists - elementwise operations (maps), filtering, etc. In many other languages, starting with Lisp but extending to many "functional" programming languages, a different style is preferred:

The idea is that if `f` is a function, then one thinks of the application
>          
    list   |---->   [ f(x) for x in list ]

on lists as a function of _two_ arguments: `f` and `list`.  The idea of viewing the function `f` as a parameter is typical in functional programming languages, and can be taken as a definition of the later term.

Some common idioms in this style, with Pythonic equivalents, are:

- `map(f, list) === [ f(x) for x in list ]`: Apply `f` element-wise to `list`.
- `filter(f, list) === [ x for x in list if f(x) ]`: Filter `list` using `f`.
- `flatMap(f, list) === [ f(x) for y in list for x in y ]`: Here `f` is a function that eats elements (of the type contained in list) and spits out lists, and `flatMap` first applies f element-wise to the elements of `list` and then _flattens_ or _concatenates_ the resulting lists.  It is sometimes also called `concatMap`.
- `reduce(f, list[, initial])`: Here `f` is a function of _two_ variables, and folds over the list applying `f` to the "accumulator" and the next value in the list.  That is, it performs the following recursion

$$    a_{-1} = \mathrm{initial} $$
$$    a_i = f(a_{i-1}, \mathrm{list}_i) $$

with the with the final answer being $a_{\mathrm{len}(\mathrm{list})-1}$.  (If initial is omitted, just start with $a_0 = \mathrm{list}_0$.)  For instance,
>           
    reduce(lambda x,y: x+y, [1,2,3,4]) = ((1+2)+3)+4 = 10
    
    
### Remark:
This is where the name "map reduce" comes from..

# Loading data into RDD

example with unstructured text



# Working with key-value pairs

## Example 3: Joins

The [Hadoop and Java_Mapreduce](Hadoop_and_Java_Mapreduce.ipynb) discuss how to implement a `join` in MR using "sum types" -- this is a data type that can consist of multiple "types" of data.  For instance, the output of an intermediate map step might have rows like
>          
    ("Business", (17, "Joe's Barbershop"))
    ("Business", (23, "Dave's Bistro"))
    ("Person", (17, "Joe Schmoe"))
    ("Person", (23, "David K. Noname"))

Fortunately for users of Spark, Spark includes a built-in `join` implementation.  From the documentation:

>         
join(otherDataset, [numTasks]):	 When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are also supported through leftOuterJoin and rightOuterJoin.




# Machine learning


import numpy as np
N = 10**6
fraction_positive = 0.5

def y(x):
    return 1 if x < fraction_positive else 0

"""
Generate Sample Data:

We're going to try to learn the rule that
y(x) = 1 if x<fraction_positive, 0 otherwise

Our training X's will go evenly from 0 to 1
"""
def generate_sample():
    sample_X = np.arange(0, 1, 1.0/N)
    np.random.shuffle( sample_X) # In-place shuffle!
    sample_Y = map(y, sample_X)
    return (sample_X, sample_Y)

(sample_X, sample_Y) = generate_sample()


## Using MLLib and it's data structures.  This is fairly quick.
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

points = ( sc.parallelize( zip(sample_X, sample_Y), 100)
             .map(lambda (x,y): LabeledPoint(y, [1, x]))
             .cache() )
model = LogisticRegressionWithSGD.train(points)

# Evaluating the model on training data
labelsAndPreds = points.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(points.count())
print "Accuracy on training set: %s" % (1 - trainErr)


## By hand.  This is the example code taken from the Spark Examples on the website.
#  This is much slower than the above code, so I'm not going to even run it (or extract predictions, or test it..)

def logistic_by_hand():
    ITERATIONS=20
    points = ( sc.parallelize( zip(sample_X, sample_Y), 20)
                 .map(lambda (x,y): LabeledPoint(y, [1, x]))
                 .cache() )
    w = np.random.ranf(size = 2) # current separating plane
    print "Original random plane: %s" % w
    for i in xrange(ITERATIONS):
        gradient = points.map(
            lambda pt: (1 / (1 + np.exp(-pt.label*(w.dot(pt.features)))) - 1) * pt.label * pt.features
        ).reduce(lambda a, b: a + b)
        w -= gradient
    print "Final separating plane: %s" % w

# logistic_by_hand()


### Exercises
1. Play with the `fraction_positive` parameter: What happens to the accuracy measure as `fraction_positive` gets below 0.30 or above 0.70? (You should be somewhat disappointed with the results!)  What do you think is happening, and can you improve on it?
1. Play with the "by hand" version (.. after lowering N to say 10**4 or so): Figure out what it's actually doing and how to use it to get results.  How much slower than the MLLib version does it seem to be?

# Building Spark applications with Scala API

Apache Spark is written in Scala. Scala (along with Python and Java) is among three languages supported by Spark, and in fact Scala functionality is typically added the first to new Spark releases.


## Preparing our first Scala Spark application

Let us start with an PySpark application we have prepared on one of the previous steps. Here it is:

In [3]:
from pyspark import SparkContext
import sys
import time

def main1(args):
    start = time.time()
    #sc = SparkContext(appName="LoadUnstructured")

    input_rdd = sc.textFile("../loading_data/unstructured/",10)
    counts = input_rdd.flatMap(lambda line: line.split()) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

    print "\nTaking the 10 most frequent words in the text and corresponding frequencies:"
    print counts.takeOrdered(10, key=lambda x: -x[1])
    end = time.time()
    print "Elapsed time: ", (end-start)

if __name__ == "__main__":
    main1(sys.argv)


Taking the 10 most frequent words in the text and corresponding frequencies:
[(u'the', 22635), (u'of', 11167), (u'and', 11086), (u'to', 10707), (u'a', 10433), (u'I', 10183), (u'in', 7006), (u'that', 6911), (u'was', 6779), (u'his', 4955)]
Elapsed time:  1.07145404816


In [ ]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.SparkContext._

object WordCount {
  def main(args: Array[String]) {

    val conf = new SparkConf().setAppName("WordCount")


    val textFile = spark.textFile("../loading_data/unstructured/",10)
    val counts = textFile.flatMap(line => line.split(" "))
                 .map(word => (word, 1))
                 .reduceByKey(_ + _)
    println("\nTaking the 10 most frequent words in the text and corresponding frequencies:")
    println(counts.takeOrdered(10).(Ordering[Int].reverse.on(x=>x._2)))
        
    val t1 = System.nanoTime()
    println("Elapsed time: " + (t1 - t0)/1000000000.)
    spark.stop()
  }
}

## Submitting Scala Spark application Q/A

Q: So you've written some Spark code in Scala. How do you submit it to Spark and run it?  
A: Use `sbt` or `maven` to package it into a Java jar, and submit it to Spark using `spark-submit`

Q: What's a Java jar?  
A: JAR (Java Archive) is a package file format typically used to aggregate many Java class files and associated metadata and resources (text, images, etc.) into one file to distribute application software or libraries on the Java platform.

### Packaging with `sbt`

**What is SBT?**  
SBT is a modern build tool written in/for Scala, though it is also a general purpose build tool  

**Why SBT?**
- Good dependency management
- Full Scala language support for creating tasks
- Launch REPL in project context

Create a root directory for your project and run:
```bash
mkdir -p src/{main,test}/{resources,scala}
mkdir lib project
```
within it. 

This script will automatically create the proper `sbt` directory structure, which borrows from the Java `maven` directory structure. The script will also generate a template `build.sbt` file at the top of the directory that you should fill out with the appropriate versions and dependencies for your app.

Then we can take our Scala code, and put it in the src folder (you should have it in the main folder, so just move it there):

```bash
mv WordCount.scala src/main/scala/
```

**Project Layout (Directory structure)**   

`project` – project definition files  
`project/build/` *yourproject* `.scala` – the main project definition file  
`project/build.properties` – project, sbt and scala version definitions  
`src/main` – your app code goes here, in a subdirectory indicating the code’s language (e.g. src/main/scala, src/main/java)  
`src/main/resources` – static files you want added to your jar (e.g. logging config)  
`src/test` – like src/main, but for tests  
`lib_managed` – the jar files your project depends on. Populated by sbt update  
`target` – the destination for generated stuff (e.g. generated thrift
code, class files, jars)  

#### `build.sbt`: Dependencies and versioning

Example `simple.sbt` (located in the root directory of your project) 

```scala
name := "WordCount"

version := "1.0"

scalaVersion := "2.10.4"

libraryDependencies ++= Seq(
    // Spark dependency
    "org.apache.spark" % "spark-core_2.10" % "1.4.1" % "provided"
)
```


#### Assembly.sbt to build a fat Jar

Example assembly.sbt located in the /project folder of your project:

```scala
resolvers += Resolver.url("artifactory", url("http://scalasbt.artifactoryonline.com/scalasbt/sbt-plugin-releases"))(Resolver.ivyStylePatterns)

addSbtPlugin("com.eed3si9n" % "sbt-assembly" % "0.13.0")
```

### Running (submitting a `jar` to Spark)

1. Run `sbt assembly` in your project's home directory. The output to console will tell you the name and location of the resulting jar (under `./target`) 

You should now see the Jar file generated:
```bash
[alexeys@bd scala_spark2] ll target/scala-2.10/
total 6968
drwxr-xr-x. 2 alexeys cses    4096 Dec  9 10:43 classes
-rw-r--r--. 1 alexeys cses 7129172 Dec  9 10:43 WordCount-assembly-1.0.jar
```

2. In the Slurm batch script, use spark-submit as usual to submit the Spark app, but you would need to specify the --class and the path to jar from the current folder, for instance:

```bash
spark-submit \
  --class "WordCount" \
  --master local[*] \
  target/scala-2.10/WordCount-assembly-1.0.jar
```